In [ ]:
import os
import sys
sys.path.append('/workspace/GREAT_ASV_system/train/')

import numpy as np
import torch


# import vox_model_bank
import model_bank
from read_data import *
from my_dataloader import *
# from sklearn.metrics import roc_curve


In [2]:
model_path = '/workspace/LOGS_OUTPUT/backup_server5/train_exp/new_pipe_test_fullvoxonly 1L sparse fixed/ckpt/vox1test_metric_saver_MINC.model'
model_id = 'Xvector_SAP_1L'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [47]:
component_dir = '/workspace/LOGS_OUTPUT/std_server5/backend/testbench'
PLDA_DIM = 370
PLDA_DATA_NAME = 'plda_data'
TEST_DATA_NAME = 'test_data_370'
PLDA_PARA_NAME = 'plda_para_370'
SCORING_PLDA_NAME = 'score_plda_370'
SCORING_COSINE_NAME = 'score_cosine_370'
# AUX_DATA_NAME = 'aux_data_370'
# SCORING_AUX_COSINE = 'score_aux_cosine_370'
# SCORING_AUX_PLDA = 'score_aux_plda_370'
PLDA_DATA_sdsv_NAME = 'plda_data_sdsv'
TEST_DATA_sdsv_NAME = 'test_data_sdsv'
PLDA_PARA_sdsv_NAME = 'plda_para_sdsv'

In [4]:
if not os.path.isdir(component_dir):
    os.makedirs(component_dir)

# GPU multiprocess

In [202]:
from multiprocessing import Process, Manager

In [203]:
num_p = 28

In [204]:
train_list = '/workspace/DATASET/std/vox_full_plda.csv'
# train_list = '/Lun0/zhiyong/dataset/plda_full_data_noVAD.csv'
# train_data = CSVDataSet(train_list)
train_data = PickleDataSet(train_list)

In [205]:
train_data_len = len(train_data)
num_per_process = (1276888 // num_p) + 1

In [206]:
manager = Manager()
class_list_new_m = manager.list()

In [207]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45580


In [208]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = model_bank.get_model(model_id, model_metric, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [ ]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

In [14]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

1276888

In [16]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [17]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

1276888

In [18]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [19]:
count

1276888

In [20]:
len(class_list_new)

7323

In [21]:
import pickle
out = component_dir+'/'+PLDA_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

In [20]:
# class_list_new = {}
# for i in class_list:
#     class_list_new[i[:-1]] = class_list[i]

# GPU multiprocess SDSV

In [5]:
from multiprocessing import Process, Manager

In [6]:
num_p = 28

In [7]:
train_list = '/workspace/DATASET/std/sdsv_full.csv'
# train_list = '/Lun0/zhiyong/dataset/plda_full_data_noVAD.csv'
# train_data = CSVDataSet(train_list)
train_data = PickleDataSet(train_list)

In [8]:
len(train_data)

85764

In [9]:
train_data_len = len(train_data)
num_per_process = (85764 // num_p) + 1

In [10]:
manager = Manager()
class_list_new_m = manager.list()

In [11]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3064
3036


In [12]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = model_bank.get_model(model_id, model_metric, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [13]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Using model bank
Using model bank
Using model bank
Using model bank
Using model bankUsing model bank

Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bankUsing model bankUsing model bankUsing model bank



Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bankUsing model bank

Using model bank
Using model bank
Using model bank
Using model bank
Using model bank


In [14]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

85764

In [15]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [16]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

85764

In [17]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [18]:
count

85764

In [19]:
len(class_list_new)

588

In [28]:
filtered_label = np.sort(list(class_list_new.keys()))[0:488]

In [30]:
class_list_new_f = {}
for i in filtered_label:
    class_list_new_f[i] = class_list_new[i]

In [34]:
import pickle
out = component_dir+'/'+PLDA_DATA_sdsv_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new_f, handle)

# PLDA_FIN

In [22]:
# import os
# import sys
# sys.path.append('./train')

# import numpy as np
# import torch


# # import vox_model_bank
# from train import train_model_new
# from train.read_data import *
# from train.my_dataloader import *
# # from sklearn.metrics import roc_curve

In [172]:
# from kaldi_plda import *
from kaldi_plda_new import *
from kaldi_lda import *
import numpy as np

In [173]:
import pickle
with open(component_dir+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [174]:
# Substract global mean
global_mean = np.zeros(512)
num_utt = 0
for count, i in enumerate(class_list_new):
    num_utt += class_list_new[i].shape[0]
    global_mean += class_list_new[i].shape[0] * np.mean(class_list_new[i], axis=0)
    
global_mean = (1.0 / num_utt) * global_mean
print('Norm of mean:', np.linalg.norm(global_mean))

Norm of mean: 1.904665238314247


In [175]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i] - global_mean

In [176]:
lda = LDA(lda_dim=PLDA_DIM, ivector_dim=512)
for i in class_list_new:
    lda.AccStats(class_list_new[i])
print('lda norm of global mean:', lda.GetGlobalMean()[1])

lda norm of global mean: 1.0221362319457326e-07


In [177]:
transform = lda.ComputeLdaTransform()

the input data has norm of mean 1.0221362319457326e-07
[7.05680318e+00 6.83084110e+00 6.49589130e+00 6.25103781e+00
 6.02699328e+00 5.93644421e+00 5.74553620e+00 5.71113328e+00
 5.52863863e+00 5.48530371e+00 5.33798738e+00 5.29171180e+00
 5.19417991e+00 5.13465376e+00 5.07992271e+00 5.03656835e+00
 4.94590764e+00 4.87222457e+00 4.86027973e+00 4.83695617e+00
 4.67446231e+00 4.62048504e+00 4.56140204e+00 4.53129694e+00
 4.49590328e+00 4.46416332e+00 4.37204092e+00 4.31959085e+00
 4.31300552e+00 4.24568159e+00 4.22761656e+00 4.13339006e+00
 4.09174010e+00 4.06669643e+00 4.00950966e+00 3.98957290e+00
 3.92033707e+00 3.90857520e+00 3.83256738e+00 3.74048687e+00
 3.67952474e+00 3.63867904e+00 3.60639986e+00 3.55344168e+00
 3.53629863e+00 3.48462131e+00 3.43491949e+00 3.41205987e+00
 3.38153426e+00 3.36127220e+00 3.34227493e+00 3.30985634e+00
 3.22345557e+00 3.20170408e+00 3.17643705e+00 3.10434679e+00
 3.08763803e+00 3.05436367e+00 3.01382095e+00 2.98352146e+00
 2.93198022e+00 2.90411634e+00

In [178]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i].dot(transform.T)

In [179]:
# normlize to sqrt(dim)
for i in class_list_new:
    scale = np.sqrt(PLDA_DIM) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
    class_list_new[i] = scale * class_list_new[i]

In [180]:
plda_stats = PldaStats(PLDA_DIM)
for i in class_list_new:
    plda_stats.add_samples(class_list_new[i])

In [181]:
plda_stats.sort()
plda_stats.is_sorted()

True

In [182]:
# test_fin_prior
plda_estimator = PldaEstimation(plda_stats)
plda_paras = plda_estimator.estimate(iteration=5)

1 5
nllr_x: 410.506519965879
nllr_y: -371.40425287899023
normalized_nllr: 39.10226708688882
nllr_m: 411.27954757668545
nllr_m_2: 411.2795475766853
part1_residual -456.1316055328956
part2_mean -411.27954757668545
normlized_obj -455.87437731835104
2 5
nllr_x: 237.2322822742737
nllr_y: -457.6529325236297
normalized_nllr: -220.42065024935602
nllr_m: 243.78477532119953
nllr_m_2: 243.78477532119962
part1_residual -438.72547415081766
part2_mean -243.7847753211998
normlized_obj -437.6074820187518
3 5
nllr_x: 234.43450562072812
nllr_y: -457.54771370630186
normalized_nllr: -223.11320808557372
nllr_m: 242.64087608072506
nllr_m_2: 242.6408760807249
part1_residual -438.7278325670568
part2_mean -242.64087608072512
normlized_obj -437.6032666048508
4 5
nllr_x: 234.15237832932453
nllr_y: -457.42389747904724
normalized_nllr: -223.2715191497227
nllr_m: 242.6170850776704
nllr_m_2: 242.6170850776702
part1_residual -438.7278292280607
part2_mean -242.6170850776704
normlized_obj -437.6031268427197
5 5
nllr_x:

In [183]:
import pickle
out = component_dir+'/'+ PLDA_PARA_NAME
with open(out, 'wb') as handle:
    pickle.dump(plda_paras, handle)

In [184]:
import pickle
with open(component_dir+'/'+ PLDA_PARA_NAME, 'rb') as handle:
    plda_paras = pickle.load(handle)

In [185]:
plda = PLDA(plda_paras[0], plda_paras[1], plda_paras[2])

In [186]:
[plda_within, plda_between] = plda_estimator.get_stat()

# PLDA_FIN_sdsv

In [22]:
# import os
# import sys
# sys.path.append('./train')

# import numpy as np
# import torch


# # import vox_model_bank
# from train import train_model_new
# from train.read_data import *
# from train.my_dataloader import *
# # from sklearn.metrics import roc_curve

In [188]:
# from kaldi_plda import *
from kaldi_plda_new import *
from kaldi_lda import *
import numpy as np

In [189]:
import pickle
with open(component_dir+'/'+PLDA_DATA_sdsv_NAME, 'rb') as handle:
    class_list_in_new = pickle.load(handle)

In [190]:
# Substract global mean
global_mean = np.zeros(512)
num_utt = 0
for count, i in enumerate(class_list_in_new):
    num_utt += class_list_in_new[i].shape[0]
    global_mean += class_list_in_new[i].shape[0] * np.mean(class_list_in_new[i], axis=0)
    
global_mean = (1.0 / num_utt) * global_mean
print('Norm of mean:', np.linalg.norm(global_mean))

Norm of mean: 12.12652709826286


In [191]:
for i in class_list_in_new:
    class_list_in_new[i] = class_list_in_new[i] - global_mean

In [192]:
lda = LDA(lda_dim=PLDA_DIM, ivector_dim=512)
for i in class_list_in_new:
    lda.AccStats(class_list_in_new[i])
print('lda norm of global mean:', lda.GetGlobalMean()[1])

lda norm of global mean: 3.223937326775059e-07


In [193]:
transform = lda.ComputeLdaTransform()

the input data has norm of mean 3.223937326775059e-07
[1.03394963e+01 9.68498237e+00 7.98811097e+00 7.62710895e+00
 7.40659819e+00 7.25867527e+00 7.04871504e+00 6.86044980e+00
 6.58582809e+00 6.31359514e+00 6.22846498e+00 6.15102178e+00
 5.99006090e+00 5.89584327e+00 5.58050782e+00 5.53644411e+00
 5.51568698e+00 5.34349933e+00 5.26714878e+00 5.24573497e+00
 5.08238761e+00 4.98427856e+00 4.91191448e+00 4.84310624e+00
 4.79134207e+00 4.70347197e+00 4.57574800e+00 4.52931257e+00
 4.35766039e+00 4.31324793e+00 4.26469621e+00 4.21468772e+00
 4.08835279e+00 4.03312136e+00 4.00315868e+00 3.87906206e+00
 3.83046421e+00 3.79815291e+00 3.71420521e+00 3.64213019e+00
 3.57440350e+00 3.50197068e+00 3.44906056e+00 3.39868806e+00
 3.30904321e+00 3.26592765e+00 3.22042961e+00 3.18824874e+00
 3.14832469e+00 3.07275323e+00 3.02925617e+00 2.98261260e+00
 2.95791561e+00 2.87306629e+00 2.84010293e+00 2.81542598e+00
 2.80527126e+00 2.73730027e+00 2.70533397e+00 2.70310971e+00
 2.61116668e+00 2.59296095e+00 

In [194]:
for i in class_list_in_new:
    class_list_in_new[i] = class_list_in_new[i].dot(transform.T)

In [195]:
# normlize to sqrt(dim)
for i in class_list_in_new:
    scale = np.sqrt(PLDA_DIM) / np.linalg.norm(class_list_in_new[i], axis=1, keepdims=True)
    class_list_in_new[i] = scale * class_list_in_new[i]

In [196]:
plda_stats = PldaStats(PLDA_DIM)
for i in class_list_in_new:
    plda_stats.add_samples(class_list_in_new[i])

In [197]:
plda_stats.sort()
plda_stats.is_sorted()

True

In [198]:
plda_paras

[array([ 2.28758791e-02, -1.92583029e-01, -5.69815745e-02,  2.48207566e-02,
         1.07089586e-01,  5.29588552e-02, -4.25924041e-02,  2.53382421e-03,
         1.88705904e-01, -2.09936139e-02,  5.14498793e-02,  3.27823750e-02,
         1.11316690e-01,  1.74073196e-01, -5.65588000e-02, -3.54532872e-02,
        -1.02041635e-02,  4.13315979e-02,  6.25163116e-02, -2.41479636e-02,
         1.05556897e-01, -1.38488886e-02, -4.00597981e-03,  3.80593547e-02,
        -1.94985378e-02, -5.84754417e-02,  2.19517969e-03, -2.70368056e-02,
        -4.78412733e-02,  1.03770087e-02, -5.15434467e-02,  1.57501991e-02,
        -1.40434899e-02, -9.01640841e-03, -3.38228523e-03,  8.63060847e-04,
         3.12018023e-02,  3.84601114e-02, -1.18589905e-02, -8.41416671e-04,
         1.26668652e-02, -1.37007597e-03, -8.48916272e-03,  6.67037226e-03,
         1.90380179e-02,  2.44943392e-02,  3.41735023e-02,  7.72534289e-03,
         1.63210721e-02, -1.10280821e-02, -1.10383094e-03, -1.35828475e-02,
         1.2

In [199]:
# test_fin_prior
plda_estimator = PldaEstimation(plda_stats)
plda_paras = plda_estimator.estimate(iteration=5)

1 5
nllr_x: 416.19672807630195
nllr_y: -374.71070634900775
normalized_nllr: 41.48602172729419
nllr_m: 416.86285387182215
nllr_m_2: 416.8628538718229
part1_residual -451.5078325732875
part2_mean -416.86285387182215
normlized_obj -451.2706247610935
2 5
nllr_x: 193.932297015006
nllr_y: -470.0382183919924
normalized_nllr: -276.10592137698643
nllr_m: 200.855603848878
nllr_m_2: 200.85560384887793
part1_residual -430.2420683777721
part2_mean -200.85560384887802
normlized_obj -428.6715013450519
3 5
nllr_x: 187.98922293711206
nllr_y: -469.7062600344786
normalized_nllr: -281.71703709736653
nllr_m: 198.14177817354323
nllr_m_2: 198.14177817354363
part1_residual -430.24139205778454
part2_mean -198.14177817354334
normlized_obj -428.6522485892615
4 5
nllr_x: 186.70345257885276
nllr_y: -469.22658878562424
normalized_nllr: -282.5231362067715
nllr_m: 197.92327260665508
nllr_m_2: 197.9232726066552
part1_residual -430.24138804357256
part2_mean -197.92327260665505
normlized_obj -428.650748535011
5 5
nllr_x

In [200]:
import pickle
out = component_dir+'/'+ PLDA_PARA_sdsv_NAME
with open(out, 'wb') as handle:
    pickle.dump(plda_paras, handle)

In [201]:
import pickle
with open(component_dir+'/'+ PLDA_PARA_sdsv_NAME, 'rb') as handle:
    plda_paras = pickle.load(handle)

In [202]:
plda_in = PLDA(plda_paras[0], plda_paras[1], plda_paras[2])

In [203]:
[plda_in_within, plda_in_between] = plda_estimator.get_stat()

# CORAL adapt

In [240]:
from CORAL_plda_adapt import *

In [241]:
coral=CORAL(plda.mean, plda_within, plda_between)

In [244]:
for i in class_list_in_new:
    for j in class_list_in_new[i]:
        coral.add_stats(1, j)

In [245]:
coral.update_plda()

In [246]:
new_coral_plda_paras = plda_estimator.get_output_from_var(coral.within_var, coral.between_var, np.squeeze(coral.mean), 370)

In [247]:
plda_new = PLDA(new_coral_plda_paras[0], new_coral_plda_paras[1], new_coral_plda_paras[2])

# CORAL plus adapt

In [259]:
from CORAL_plus_plda_adapt import *

In [260]:
coralplus=CORALPlus(plda.mean, plda_within, plda_between)

In [261]:
for i in class_list_in_new:
    for j in class_list_in_new[i]:
        coralplus.add_stats(1, j)

In [262]:
coralplus.update_plda()

In [263]:
new_coral_plus_plda_paras = plda_estimator.get_output_from_var(coralplus.within_var, coralplus.between_var, np.squeeze(coralplus.mean), 370)

In [264]:
plda_new = PLDA(new_coral_plus_plda_paras[0], new_coral_plus_plda_paras[1], new_coral_plus_plda_paras[2])

# LIP

In [276]:
from LIP_plda_adapt import *

In [121]:
# plda_in = plda
# plda_in_within = plda_within
# plda_in_between = plda_between

In [277]:
lip = LIP()
lip.interpolation(plda_in.mean, plda_in_within, plda_in_between, plda_within, plda_between)

In [278]:
new_lip_plda_paras = plda_estimator.get_output_from_var(lip.within_var, lip.between_var, np.squeeze(lip.mean), 370)

In [279]:
plda_new = PLDA(new_lip_plda_paras[0], new_lip_plda_paras[1], new_lip_plda_paras[2])

# CIP

In [294]:
from CIP_plda_adapt import *
from CORAL_plda_adapt import *

In [295]:
coral=CORAL(plda.mean, plda_within, plda_between)

In [296]:
for i in class_list_in_new:
    for j in class_list_in_new[i]:
        coral.add_stats(1, j)

In [297]:
coral.update_plda()

In [137]:
# plda_in = plda
# plda_in_within = plda_within
# plda_in_between = plda_between

In [298]:
cip=CIP()
cip.interpolation(plda_in.mean, plda_in_within, plda_in_between, coral)

In [299]:
new_cip_plda_paras = plda_estimator.get_output_from_var(cip.within_var, cip.between_var, np.squeeze(cip.mean), 370)

In [300]:
plda_new = PLDA(new_cip_plda_paras[0], new_cip_plda_paras[1], new_cip_plda_paras[2])

# LIP-REG

In [312]:
from LIP_REG_plda_adapt import *

In [154]:
# plda_in = plda
# plda_in_within = plda_within
# plda_in_between = plda_between

In [313]:
lipreg = LIPReg()
lipreg.interpolation(plda_in.mean, plda_in_within, plda_in_between, plda_within, plda_between)

In [314]:
new_lipreg_plda_paras = plda_estimator.get_output_from_var(lipreg.within_var, lipreg.between_var, np.squeeze(lipreg.mean), 370)

In [315]:
plda_new = PLDA(new_lipreg_plda_paras[0], new_lipreg_plda_paras[1], new_lipreg_plda_paras[2])

# CIP-REG

In [327]:
from CIP_REG_plda_adapt import *
from CORAL_plda_adapt import *

In [171]:
coral=CORAL(plda.mean, plda_within, plda_between)

In [172]:
for i in class_list_in_new:
    for j in class_list_in_new[i]:
        coral.add_stats(1, j)

In [173]:
coral.update_plda()

In [174]:
# plda_in = plda
# plda_in_within = plda_within
# plda_in_between = plda_between

In [328]:
cipreg=CIPReg()
cipreg.interpolation(plda_in.mean, plda_in_within, plda_in_between, coral)

In [329]:
new_cipreg_plda_paras = plda_estimator.get_output_from_var(cipreg.within_var, cipreg.between_var, np.squeeze(cipreg.mean), 370)

In [330]:
plda_new = PLDA(new_cipreg_plda_paras[0], new_cipreg_plda_paras[1], new_cipreg_plda_paras[2])

# Unsupervised adaptation

In [342]:
from kaldi_plda_new import *

In [343]:
adaptor = PldaUnsupervisedAdaptor(dim=PLDA_DIM)

In [346]:
for i in class_list_in_new:
    for j in class_list_in_new[i]:
        adaptor.add_stats(j)

In [348]:
import pickle
with open(component_dir+'/'+ PLDA_PARA_NAME, 'rb') as handle:
    plda_paras = pickle.load(handle)

In [350]:
new_kaldi_uda_plda_paras = adaptor.update_plda(plda_paras[0], plda_paras[1], plda_paras[2])

In [351]:
plda_new = PLDA(new_kaldi_uda_plda_paras[0], new_kaldi_uda_plda_paras[1], new_kaldi_uda_plda_paras[2])

# Evaluation

In [20]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = '/workspace/DATASET/std/vox1test_trial_data.csv'
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, None)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500


In [21]:
count+1

4874

In [22]:
len(test_list)

4874

In [23]:
import pickle
out = component_dir+'/'+TEST_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# Evaluation sdsv

In [50]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = '/workspace/DATASET/std/sdsvsmall_trial_data.csv'
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, None)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000


In [51]:
count+1

14490

In [52]:
len(test_list)

14490

In [53]:
import pickle
out = component_dir+'/'+TEST_DATA_sdsv_NAME
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# SDSV Scoring PLDA kaldi-uda

In [352]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [353]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [354]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [355]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [356]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [357]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [358]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [359]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [360]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [361]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.0332815734989648, 0.2934955141476865, 0.5019927536231884)
Starting point for CLLR is 0.401168
Converged linear model with loss 0.13812855503534988
(0.0332815734989648, 0.2934955141476865, 0.3532608695652174)


In [362]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA CIP-REG

In [331]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [332]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [333]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [334]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [335]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [336]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [337]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [338]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [339]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [340]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.030296756383712906, 0.28263457556935445, 0.3606625258799172)
Starting point for CLLR is 0.594836
Converged linear model with loss 0.12833518508854863
(0.030296756383712906, 0.28263457556935445, 0.35998102139406485)


In [341]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA LIP-REG

In [316]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [317]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [318]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [319]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [320]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [321]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [322]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [323]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [324]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [325]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.028433402346445823, 0.2858523119392705, 0.3444703243616287)
Starting point for CLLR is 0.641845
Converged linear model with loss 0.12459125133976001
(0.028433402346445823, 0.2858523119392705, 0.380891994478951)


In [326]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA CIP

In [301]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [302]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [303]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [304]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [305]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [306]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [307]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [308]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [309]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [310]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02969289164941339, 0.2835144927536276, 0.3795462387853692)
Starting point for CLLR is 0.528276
Converged linear model with loss 0.12595929005448164
(0.02969289164941339, 0.2835144927536276, 0.3566856452726018)


In [311]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA LIP

In [280]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [281]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [282]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [283]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [284]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [285]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [286]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [287]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [288]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [289]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02789855072463768, 0.2832125603864702, 0.3481280193236715)
Starting point for CLLR is 0.602749
Converged linear model with loss 0.12344965045191823
(0.02789855072463768, 0.2832125603864702, 0.3775534851621808)


In [290]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA CORAL+

In [265]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [266]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [267]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [268]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [269]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [270]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [271]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [272]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [273]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [274]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02789855072463768, 0.28452380952380985, 0.3773895790200138)
Starting point for CLLR is 0.486375
Converged linear model with loss 0.12022658409028258
(0.02789855072463768, 0.28452380952380985, 0.34453071083505865)


In [275]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA CORAL

In [248]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [249]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [250]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [251]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [252]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [253]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [254]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [255]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [256]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [257]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.0296583850931677, 0.2910282953761233, 0.40492581090407176)
Starting point for CLLR is 0.493019
Converged linear model with loss 0.12652412044941985
(0.0296583850931677, 0.2910282953761233, 0.3575224292615597)


In [258]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA OUT

In [158]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [159]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [160]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda.transform_ivector(test_list[i], 1)

In [161]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [162]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [163]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [164]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [165]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [166]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [167]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02629399585921325, 0.2928312629399539, 0.385248447204969)
Starting point for CLLR is 0.552906
Converged linear model with loss 0.11912664726358842
(0.02629399585921325, 0.2928312629399539, 0.37501725327812285)


In [141]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.04773982056590753, 0.48993271221532486, 9.678761214630779)
Starting point for CLLR is 0.890078
Converged linear model with loss 0.19103426899687426
(0.04773982056590753, 0.48993271221532486, 0.5256297446514838)


In [142]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# SDSV Scoring PLDA IN

In [205]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [206]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [207]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_in.transform_ivector(test_list[i], 1)

In [208]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_in.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [47]:
# Cosine Scoring

In [209]:
trail_path = '/workspace/DATASET/std/sdsvsmall_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [210]:
import pickle
with open(component_dir+'/'+TEST_DATA_sdsv_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [211]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [212]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [213]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [216]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.03012422360248447, 0.2496204278813, 0.12603174603174602)
Starting point for CLLR is 0.625024
Converged linear model with loss 0.13291985466567877
(0.03012422360248447, 0.2496204278813, 0.13453071083505866)


In [215]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/sdsvsmall_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.049189095928226366, 0.4170721187025469, 1.0)
Starting point for CLLR is 0.869133
Converged linear model with loss 0.18979610822148688
(0.049189095928226366, 0.4170721187025469, 1.0)


# Scoring PLDA

In [25]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [26]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [27]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda.transform_ivector(test_list[i], 1)

In [28]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [47]:
# Cosine Scoring

In [30]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [31]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [32]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [33]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [34]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [35]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02104984093319194, 0.26163838812301365, 0.4125662778366914)
Starting point for CLLR is 0.326006
Converged linear model with loss 0.08623393210098965
(0.02104984093319194, 0.26163838812301365, 0.2688229056203606)


In [36]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# scoring PLDA CORAL

In [71]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [72]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [75]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [76]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [47]:
# Cosine Scoring

In [77]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [78]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [79]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [80]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [81]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [82]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02142099681866383, 0.2611611876988343, 0.3741781548250265)
Starting point for CLLR is 0.347226
Converged linear model with loss 0.08987157382109044
(0.02142099681866383, 0.2611611876988343, 0.291118769883351)


In [83]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# scoring PLDA CORAL+

In [107]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [108]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [109]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [110]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [111]:
# Cosine Scoring

In [112]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [113]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [114]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [115]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [116]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [117]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.020784729586426298, 0.2615588547189809, 0.4023594909862142)
Starting point for CLLR is 0.308798
Converged linear model with loss 0.08617091907281849
(0.020784729586426298, 0.2615588547189809, 0.2796129374337222)


In [118]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# scoring PLDA LIP

In [125]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [126]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [127]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [128]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [111]:
# Cosine Scoring

In [129]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [130]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [131]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [132]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [133]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [134]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02104984093319194, 0.26163838812301365, 0.4125662778366914)
Starting point for CLLR is 0.326006
Converged linear model with loss 0.08623393210098967
(0.02104984093319194, 0.26163838812301365, 0.2688229056203606)


In [135]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# scoring PLDA CIP

In [141]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [142]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [143]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [144]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [145]:
# Cosine Scoring

In [146]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [147]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [148]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [149]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [150]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [151]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02131495227995761, 0.26216861081654635, 0.4028101802757158)
Starting point for CLLR is 0.331509
Converged linear model with loss 0.08786970159803127
(0.02131495227995761, 0.26216861081654635, 0.28467656415694587)


In [152]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# scoring PLDA LIP-REG

In [158]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [159]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [160]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [161]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [162]:
# Cosine Scoring

In [163]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [164]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [165]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [166]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [167]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [168]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02104984093319194, 0.26163838812301365, 0.4125662778366914)
Starting point for CLLR is 0.326006
Converged linear model with loss 0.08623393210098963
(0.02104984093319194, 0.26163838812301365, 0.2688229056203606)


In [169]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# scoring PLDA CIP-REG

In [179]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [180]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [181]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda_new.transform_ivector(test_list[i], 1)

In [182]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda_new.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [183]:
# Cosine Scoring

In [184]:
trail_path = '/workspace/DATASET/std/vox1test_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [185]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [186]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [187]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [188]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [189]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.020890774125132554, 0.2638123011664924, 0.41362672322375393)
Starting point for CLLR is 0.314537
Converged linear model with loss 0.08645988000511412
(0.020890774125132554, 0.2638123011664924, 0.2739395546129374)


In [190]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/std/vox1test_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# JB

In [68]:
from JB import *
import numpy as np

/home/dell/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [69]:
import pickle
with open(component_dir+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [70]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])

In [71]:
train_list = np.zeros((1276888, 512))
label = []
last_pos = 0
for count, i in enumerate(class_list_new):
    this_len = class_list_new[i].shape[0]
    train_list[last_pos:last_pos+this_len] = class_list_new[i]
    last_pos = last_pos+this_len
    label = label + this_len * [i]
    if (count+1) % 1000 == 0:
        print(count+1)

1000
2000
3000
4000
5000
6000
7000


In [75]:
train_list.shape

(1276888, 512)

In [76]:
len(label)

1276888

In [77]:
global_mean = np.mean(train_list, axis=0)

In [81]:
np.linalg.norm(global_mean)

1.9046652415934415

In [82]:
train_list_centered = train_list - global_mean

In [83]:
A, G = JointBayesian_Train(train_list_centered, label, fold = component_dir, complete_stats=True)

(1276888, 512)
prepare done, maxNumberInOneClass= 1002
Iterations-0: 0.0010711645996589437
Iterations-1: 1.3523100215732059e-05
Iterations-2: 2.4056012775033254e-06
Iterations-3: 7.342351828003286e-07
Iterations-4: 2.8068888850328114e-07
Iterations-5: 1.184689619041678e-07
Iterations-6: 5.267479095722711e-08
Iterations-7: 2.4178416589029948e-08
Iterations-8: 1.1338391238278522e-08
Iterations-9: 5.399002417915827e-09
Iterations-10: 2.6014852098979146e-09
Iterations-11: 1.2648995479417483e-09
Iterations-12: 6.188518191301809e-10
Iterations-13: 3.0677699544240897e-10
Iterations-14: 1.5350548541958987e-10
Iterations-15: 8.009531443608684e-11
Iterations-16: 4.7031017931828334e-11
Iterations-17: 3.413629428325939e-11
Iterations-18: 3.0268280274376596e-11
Iterations-19: 2.927871093020977e-11


In [84]:
trail_path = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
score_out_path = component_dir+'/'+'JB_score'

In [85]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [86]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = transform.dot(test_list[i])
#     test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
#     test_list[i] = plda.transform_ivector(test_list[i], 1)

In [88]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = Verify(A, G, enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [89]:
score_file = component_dir+'/'+'JB_score'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.028791092258748652, 0.2808589607635198, 0.592099681866384)
Starting point for CLLR is 10.192940
(0.028791092258748652, 0.2808589607635198, 0.592099681866384)


# LDA-JB

In [115]:
from JB_tmp import *
import numpy as np

In [116]:
import pickle
with open(component_dir+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [117]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])

In [118]:
# Substract global mean
global_mean = np.zeros(512)
num_utt = 0
for count, i in enumerate(class_list_new):
    num_utt += class_list_new[i].shape[0]
    global_mean += class_list_new[i].shape[0] * np.mean(class_list_new[i], axis=0)
    
global_mean = (1.0 / num_utt) * global_mean
print('Norm of mean:', np.linalg.norm(global_mean))

Norm of mean: 1.9046652386925422


In [119]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i] - global_mean

In [120]:
PLDA_DIM = 512

In [121]:
lda = LDA(lda_dim=PLDA_DIM, ivector_dim=512)
for i in class_list_new:
    lda.AccStats(class_list_new[i])
print('lda norm of global mean:', lda.GetGlobalMean()[1])

lda norm of global mean: 1.0363030848906236e-07


In [122]:
transform = lda.ComputeLdaTransform()

the input data has norm of mean 1.0363030848906236e-07
[7.05680318e+00 6.83084110e+00 6.49589130e+00 6.25103781e+00
 6.02699328e+00 5.93644421e+00 5.74553620e+00 5.71113328e+00
 5.52863863e+00 5.48530371e+00 5.33798738e+00 5.29171180e+00
 5.19417991e+00 5.13465376e+00 5.07992271e+00 5.03656835e+00
 4.94590764e+00 4.87222457e+00 4.86027973e+00 4.83695617e+00
 4.67446231e+00 4.62048504e+00 4.56140204e+00 4.53129694e+00
 4.49590328e+00 4.46416332e+00 4.37204092e+00 4.31959085e+00
 4.31300552e+00 4.24568159e+00 4.22761656e+00 4.13339006e+00
 4.09174010e+00 4.06669643e+00 4.00950966e+00 3.98957290e+00
 3.92033707e+00 3.90857520e+00 3.83256738e+00 3.74048687e+00
 3.67952474e+00 3.63867904e+00 3.60639986e+00 3.55344168e+00
 3.53629863e+00 3.48462131e+00 3.43491949e+00 3.41205987e+00
 3.38153426e+00 3.36127220e+00 3.34227493e+00 3.30985634e+00
 3.22345557e+00 3.20170408e+00 3.17643705e+00 3.10434679e+00
 3.08763803e+00 3.05436367e+00 3.01382095e+00 2.98352146e+00
 2.93198022e+00 2.90411634e+00

In [123]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i].dot(transform.T)

In [124]:
# normlize to sqrt(dim)
for i in class_list_new:
    scale = np.sqrt(PLDA_DIM) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
    class_list_new[i] = scale * class_list_new[i]

In [125]:
train_list = np.zeros((1276888, PLDA_DIM))
label = []
last_pos = 0
for count, i in enumerate(class_list_new):
    this_len = class_list_new[i].shape[0]
    train_list[last_pos:last_pos+this_len] = class_list_new[i]
    last_pos = last_pos+this_len
    label = label + this_len * [i]
    if (count+1) % 1000 == 0:
        print(count+1)

1000
2000
3000
4000
5000
6000
7000


In [126]:
train_list.shape

(1276888, 512)

In [127]:
len(label)

1276888

In [128]:
A, G = JointBayesian_Train(train_list, label, fold = component_dir, complete_stats=True)

(1276888, 512)
prepare done, maxNumberInOneClass= 1002
Iterations-0: 0.0013375771781835597
Iterations-1: 0.00020715537950710702
Iterations-2: 8.002533716710512e-05
Convergence:  2 8.002533716710512e-05


In [129]:
trail_path = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
score_out_path = component_dir+'/'+'JB_lda_512_score'

In [130]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [131]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]

In [132]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = Verify(A, G, enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [110]:
score_file = component_dir+'/'+'JB_lda_score'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.020837751855779407, 0.26558854718982505, 0.5770943796394485)
Starting point for CLLR is 10.179847
(0.020837751855779407, 0.26558854718982505, 0.5770943796394485)


In [133]:
score_file = component_dir+'/'+'JB_lda_512_score'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.01993637327677622, 0.26134676564156306, 0.5799575821845175)
Starting point for CLLR is 10.246580
(0.01993637327677622, 0.26134676564156306, 0.5799575821845175)
